In [ ]:
import numpy as np
import pandas as pd
from source.Files_operating import read_sht_data

In [ ]:
F_ID = 44184
dir_path = "C:/Users/f.belous/Work/Projects/Plasma_analysis/data/sht/G-ELM/"  
# D:/Edu/Lab/Projects/Plasma_analysis/data/sht/G-ELM/ | C:/Users/f.belous/Work/Projects/Plasma_analysis/data/sht/G-ELM/

df = read_sht_data(f'sht{F_ID}', dir_path)
df = df.rename(columns={"ch1": "d_alpha"})

df["mark_dELM"] = 0
df["mark_LCO"] = 0
df["mark_EHO"] = 0

df.describe()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

def get_d2_peaks(diff_data: np.array, mark_data: np.array, s_i: int) -> np.array:
    l_ = 0
    fr_fl = False
    peaks_ind = []
    for i in range(mark_data.shape[0]):
        if mark_data[i] == 1 and not fr_fl:
            fr_fl = True
            l_ = i
        elif fr_fl and mark_data[i] == 0:
            fr_fl = False
            d2_data = np.diff(diff_data)
            res = get_peaks(d2_data[l_-10:i+30], s_i) + l_-10
            peaks_ind += res.tolist()
    peaks_ind = np.array(peaks_ind)
    return peaks_ind, d2_data[peaks_ind]

mode_plotting = 0  # int(input("Input visualising mode [ 0 - manual | 1 - only marked ]:"))
# mode_marking = int(input("Input marking mode (0 - manual | 1 - semiauto): ").strip().split()[0])
l_edge = input(f"Input start index [0:{df.shape[0]//1000*1000}]: ").strip().split()[0]

if l_edge == "":
    l_edge = 0
else:
    l_edge = int(l_edge)


# print("---")
step = 2000
width = 3000

step_away = 10

while l_edge < df.shape[0]:
    r_edge = min(l_edge + width, df.shape[0] - 1)
    
    if mode_plotting == 1 and sum(mark_d_alpha[l_edge:r_edge]) == 0 and sum(mark_sxr[l_edge:r_edge]) == 0:
        l_edge += step
        continue
    
    fig, axs = plt.subplots(nrows=1, sharex=True, gridspec_kw={'hspace': 0.05})

    fig.set_figwidth(16)
    fig.set_figheight(4)
    
    axs[0].set_title(f"#{F_ID}")

    axs[0].plot(range(l_edge, r_edge), df.d_alpha[l_edge:r_edge], label="D-alpha")
    axs[0].plot(range(l_edge, r_edge), df.mark_dELM[l_edge:r_edge], label="Mark dELM", color="green")
    axs[0].plot(range(l_edge, r_edge), df.mark_LCO[l_edge:r_edge], label="Mark LCO", color="blue")
    axs[0].plot(range(l_edge, r_edge), df.mark_EHO[l_edge:r_edge], label="Mark EHO", color="orange")
    axs[0].grid(which='major', color='#DDDDDD', linewidth=0.9)
    axs[0].grid(which='minor', color='#DDDDDD', linestyle=':', linewidth=0.7)
    axs[0].minorticks_on()
    axs[0].xaxis.set_minor_locator(AutoMinorLocator(10))
    axs[0].legend()
    
    plt.show()

    res = list(map(int, input(f"Input index pairs of fragments ({df.t[l_edge]} - {df.t[r_edge]} ms):\n").strip().split()))
    for i in range(0, len(res), 2):
        board_ind = [res[i], res[i+1]]

        mark = float(input("Input mark to set [ 0 | 1 | 2 ]: "))
        
        if mode_marking and mark > 0:
            board_ind = get_borders(df.loc[res[i]:res[i + 1], "ch1"].to_numpy(), scale=1.5)
            board_ind[0] = max(board_ind[0] - step_away + res[i], 0)
            board_ind[1] += res[i]
        
        df.loc[board_ind[0]:board_ind[1], "ch1_marked"] = mark

    plt.close()
    plt.clf()
    
    new_l_edge = input(f"Input start index [0:{df.shape[0]//1000*1000}]: ").strip().split()[0]
    
    if new_l_edge == "":
        l_edge += step
    else:
        l_edge = int(l_edge)

# 217500